In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
from UtilityFunctions import *
from datapreprocessing import *
from implementations import *
from patternsmissingvalues import *

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

# Exploring the data

In [ ]:
y.shape,tX.shape,ids.shape


In [ ]:
tX[0]

In [ ]:
(tX[:,0].shape,np.std(tX[:,0]))

In [ ]:
plt.scatter(tX[:,0],y, marker=".", color='b')

Ne sont pas des outliers, mais contiennent juste des missing values à -999

## Correlation Matrix:

In [ ]:
#corrcoef prod des vals normalisées à partir de la mat de correlation
cov = np.cov(tX.T)
#print(cov.shape, np.max(cov))
corr = np.corrcoef((tX.T))
#print(corr.shape, (corr))
plt.imshow(corr, cmap='hot')
res = np.where(corr > 0.95) #arbitrary threshold for strong correlation
listOfCoordinates= list(zip(res[0], res[1]))
listOfCoordinates = [cord for cord in listOfCoordinates if cord[0] != cord[1]] #remove diagonal
listOfCoordinates = {tuple(sorted(t)): t for t in listOfCoordinates}#remove commutative elements
for cord in listOfCoordinates:
        print(cord)

We identify strong correlations between the features listed above

## PCA:

In [ ]:
X_pca = pca(tX)
plt.scatter(X_pca[:,0],y, marker=".", color='b')

Plot of the first PCA component against the output

In [ ]:
import matplotlib.cm as cm # To colour dots of scatter plot
colors = cm.rainbow(y)
plt.scatter(X_pca[:,0],X_pca[:,1], marker=".", color=colors)

Plot of the first PCA component against the second PCA component. The dots are color labeled according to their output value y. Unfortunately, we notice that the clusters identified in the plot do not discriminate between the 2 output categories.

## Missing values positions :

In [ ]:
tX2 = tX.copy()
res = []
for el in tX2.T:
    res.append(el[el > -999])

mu = [np.mean(el) for el in res]
sigma = [np.std(el) for el in res]

for col, mu1, sigma1 in zip(tX2.T,mu,sigma):
    col[col == -999] = np.random.normal(mu1, sigma1, np.sum([col == -999][0]))
    
print(np.sum(tX==-999), np.sum(tX2==-999))

## Distributions of the variables (w/o missing values) :

In [ ]:
missing_val = np.zeros(tX.shape)
missing_val[tX==-999] = 1
total_cols = np.sum(missing_val, axis=0)/y.shape
total_rows = np.sum(missing_val, axis=1)/np.shape(tX)[1]
tX_reduced = tX[: ,total_cols < 0.5] #select only cols where less than 50% val missing and < 30% for rows
tX_reduced = tX_reduced[total_rows<0.3, :] #and < 30% for rows
y_reduced = y[total_rows<0.3]

fig,ax = plt.subplots(5, 6, sharex='col')
for i in range(tX.shape[1]):
    tXcol = [el for el in tX[:,i] if el > -999]  #keep only values in the normal range for the plot
    ax[i%5,i%6].hist(tXcol, bins=40, log=True)
fig.set_figheight(150)
fig.set_figwidth(150)
plt.show

# Global data preprocessing :

In [ ]:
x, mean_x, std_x = standardize(tX)
tX = adding_offset(x)
xtrain,ytrain,xtest,ytest=split_data(tX,y,0.9)
#pX=build_poly(tX, 5) #-- marche pas --> parce que la fonction est prévu que pour le cas 1D
pX= add_higher_degree_terms(tX,3)
xtrainpol,ytrainpol,xtestpol,ytestpol=split_data(pX, y, 0.9) #on polynomial version,with ration= 0.9 and default seed=1

# Global implementations

## Least squares gradient descent :

In [ ]:
initial_w = np.zeros(np.shape(tX)[1])
gamma = 0.0825
max_iters = 1000
final_w_gd, final_loss_gd = least_squares_GD(y, tX, initial_w, max_iters, gamma)

## Least squares stochastic gradient descent :

In [ ]:
gamma = 0.005
max_iters = 500
final_w_sgd, final_loss_sgd = least_squares_SGD(y, tX, initial_w, max_iters, gamma)

## Least squares with normal equation  

In [ ]:
w_ne,loss_ne= least_squares(y, tX)

#### on polynomial version of the data set :

In [ ]:
w_ne_pol, loss_ne_pol = least_squares(y, pX)
# tester ici quel degré est le mieux pour pX

#### and with split of test and train data :

In [ ]:
w_ne_pol_train, loss_ne_pol_train=least_squares(ytrainpol, xtrainpol)
rmse_ne_pol_train=rmse(loss_ne_pol_train)
loss_ne_pol_test= compute_loss(ytestpol, xtestpol, w_ne_pol_train)
rmse_ne_pol_test=rmse(loss_ne_pol_test)
print(" Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(tr=rmse_ne_pol_train, te=rmse_ne_pol_test))

## Ridge regression :

In [ ]:
lambdas = np.logspace(-5, 0, 15)
rmse_rr = []
for ind, lambda_ in enumerate(lambdas):
        w_rr,loss_rr = ridge_regression(y, tX, lambda_)
        rmse_rr.append(rmse(loss_rr))

plot_implementation(rmse_rr, lambdas)

#### on polynomial version of the data set :

In [ ]:
lambdas = np.logspace(-5, 0, 15)
rmse_rr_pol = []
for ind, lambda_ in enumerate(lambdas):
        w_rr_pol,loss_rr_pol = ridge_regression(y, pX, lambda_)
        rmse_rr_pol.append(rmse(loss_rr_pol))

plot_implementation(rmse_rr_pol, lambdas)

#### and with split of test and train data :

In [ ]:
lambdas = np.logspace(-5, 0, 15)
rmse_rr_pol_train = []
rmse_rr_pol_test = []
for ind, lambda_ in enumerate(lambdas):
        w_rr_pol,loss_rr_pol_train = ridge_regression(ytrainpol,xtrainpol , lambda_)
        rmse_rr_pol_train.append(rmse(loss_rr_pol_train))
        rmse_rr_pol_test.append(rmse(compute_loss(ytestpol,xtestpol,w_rr_pol)))
plot_train_test(rmse_rr_pol_train, rmse_rr_pol_test, lambdas)
#JEROME: plot a l'air bizarre, exactement meme erreur pour test et train

## Log likelihood

In [ ]:
#### Example - base mode
gamma=0.5
ws_1,log_likelihoods_1 = logistic_regression(rescale_y(y), tX, np.random.rand(tX.shape[1])/1000000,10, gamma)
plt.scatter(rescale_predictions(compute_p(ws_1[-1],tX))[1:1000],y[1:1000], marker=".", color='b')

In [ ]:
ws_2, log_likelihoods_2 = logistic_regression(rescale_y(y), tX, np.random.rand(tX.shape[1])/1000000,10,gamma)
plt.scatter(rescale_predictions(compute_p(ws_2[-1],tX))[1:1000],y[1:1000], marker=".", color='b')
np.linalg.norm(ws_1[-1]-ws_2[-1])

#### Optimizations

In [ ]:

##### Adding w0 to the model

tX_w0 = adding_offset(tX)

print(tX_w0[0:5,0:5],tX_w0.shape)

ws_3,log_likelihoods_3 = logistic_regression(rescale_y(y), tX_w0, np.random.rand(tX_w0.shape[1])/1000000,10)

plt.scatter(rescale_predictions(compute_p(ws_3[-1],tX_w0))[1:1000],y[1:1000], marker=".", color='b')

ws_4,log_likelihoods_4 = logistic_regression(rescale_y(y), tX_w0, np.random.rand(tX_w0.shape[1])/1000000,10)

plt.scatter(rescale_predictions(compute_p(ws_4[-1],tX_w0))[1:1000],y[1:1000], marker=".", color='b')

np.linalg.norm(ws_3[-1]-ws_4[-1])

## Split train and test datas

# Outliers Management

##### replace -999 by average value

In [ ]:
tX_corr = set_missing_explanatory_vars_to_mean(tX)

In [ ]:
tX_corr[:,1:10]

In [ ]:
tX_corr_pca = pca(tX_corr)

In [ ]:
colors = cm.rainbow(y)
plt.scatter(tX_corr_pca[:,0],tX_corr_pca[:,1], marker=".", color=colors)

In [ ]:
colors = cm.rainbow(y[1:5000])
plt.scatter(tX_corr_pca[1:5000,0],tX_corr_pca[1:5000,1], marker=".", color=colors)

##### Removing outliers

In [ ]:
#find and delete outlier

ind = np.arange(len(tX_corr_pca[:,0]))
ind_outliers = ind[np.where(tX_corr_pca[:,0] < -50)] # find a suitable criterion, 
y_cleared = np.delete(y,ind_outliers)
tX_corr_cleared = np.delete(tX_corr_pca,ind[np.where(tX_corr_pca[:,0] < -50)],axis=0)

In [ ]:
print(y_cleared.shape,tX_corr_cleared.shape)

In [ ]:
tX_corr_cleared_w0 = adding_offset(tX_corr_cleared)

In [ ]:
ws_5,log_likelihoods_5 = logistic_regression(rescale_y(y_cleared), tX_corr_cleared_w0, np.random.rand(tX_corr_cleared_w0.shape[1])/1000000,10,gamma)

In [ ]:
plt.scatter(rescale_predictions(compute_p(ws_5[-1],tX_corr_cleared_w0))[1:1000],y_cleared[1:1000], marker=".", color='b')

In [ ]:
ws_6, log_likelihoods_6 = logistic_regression(rescale_y(y_cleared), tX_corr_cleared_w0, np.random.rand(tX_corr_cleared_w0.shape[1])/1000000,10,gamma)

In [ ]:
plt.scatter(rescale_predictions(compute_p(ws_6[-1],tX_corr_cleared_w0))[1:1000],y_cleared[1:1000], marker=".", color='b')

In [ ]:
np.linalg.norm(ws_5[-1]-ws_6[-1])

In [ ]:
np.linalg.norm(ws_3[-1]-ws_5[-1])

In [ ]:
print(ws_3[-1],ws_5[-1])

# Taking into account patterns for the handling of missing values

In [ ]:
DATA_TEST_PATH = "../data/test.csv/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
DATA_TRAIN_PATH = "../data/train.csv/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# Test that in both test and train data the same columns are full of gaps
tX_mv = sum(tX == -999) # Array with columnwise count of faulty measurements in training data
tX_mv[tX_mv > 0] = 1 
tX_test_mv = sum(tX_test == -999) # Array with columnwise count of faulty measurements in test data
tX_test_mv[tX_test_mv > 0] = 1

np.linalg.norm(tX_mv-tX_test_mv) # = 0 means both training and test data have values missing only in the excact same explanatory variables.

In [ ]:
# Missing values don't appear to be random
tX_mv = sum(tX == -999)
tX_test_mv = sum(tX_test == -999)

print(tX_mv,tX_test_mv) # Arrays with columnwise counts of faulty measurements in training and test data

In [ ]:
ind = np.arange(tX.shape[1])
tX_col_mv = tX[:,ind[np.where(tX_mv > 0)]]
test = np.zeros(tX_col_mv.shape)
test[np.where(tX_col_mv == -999)] = 1
print(test[1:10,:]) # Extracted coulums that contain -999s. '1' represents missing value, '0' otherwise.

In [ ]:
np.flip(np.power(10,np.arange(test.shape[1]),dtype=np.int64)) # Sleight of hand: matrix multiplication  with the following array.

In [ ]:
np.dot(test,np.flip(np.power(10,np.arange(test.shape[1]),dtype=np.int64))) # The pattern of missing columns is represented by an array of numbers representing all samples in the usual order. The numbers are binary code for the matrix of the extracted columns just above.

In [ ]:
np.unique(np.dot(test,np.power(10,np.arange(test.shape[1]),dtype=np.int64))) # All the different configurations of the missing values. There are only six of them.

#### Pattern of missing values and logistic regession

In [ ]:
# Splitting the data...
tX_split, ind_row_groups, groups_mv_num = split_data_according_to_pattern_of_missing_values(tX) # => regroupement of the steps performed above
y_split = split_y_according_to_pattern_of_missing_values(y, ind_row_groups)

'''import matplotlib.cm as cm

for tX_group, y_group in zip(tX_split,y_split):
    tX_group_pca = pca(tX_group)
    colors = cm.rainbow(y_group)
    plt.scatter(tX_group_pca[:,0],tX_group_pca[:,1], marker=".", color=colors)'''

# ... and finding each subgroup's the optimal weights as well as predicting the y's of the training set using linear regression
#     => condensed version is the function 'find_optimal_weights_pattern_mv'

ws_groups = [] # list to stock weights of subgroups
y_pred = np.zeros([len(y)]) # initialize array y_pred

for tX_group, y_group, ind_row_group in zip(tX_split,y_split,ind_row_groups) :
    #print(tX_group, y_group, ind_row_group)
    
    # All-zero columns appear in the reduced data sets. They interfere with the calculation of the jacobean matrix.
    # The corresponding weights are set to zero and the other weights are determined using a further reduced dataset without those columns.
    
    ind_col_non_zero = np.arange(len(tX_group[0,:]))[sum(tX_group**2)>0]
    ws_1,log_likelihoods_1 = logistic_regression(rescale_y(y_group), tX_group[:,ind_col_non_zero], np.random.rand(tX_group[:,ind_col_non_zero].shape[1])/1000000,10,gamma)
    ws_2,log_likelihoods_2 = logistic_regression(rescale_y(y_group), tX_group[:,ind_col_non_zero], np.random.rand(tX_group[:,ind_col_non_zero].shape[1])/1000000,10,gamma)
    assert np.allclose(ws_1[-1], ws_2[-1]) # The weights are calculated twice with different initial ws => asserting that they are the same.
    w = np.zeros(len(tX_group[0,:]))
    w[ind_col_non_zero] = ws_1[-1]
    ws_groups.append(w)
    y_pred[ind_row_group] = rescale_predictions(compute_p(w,tX_group))

plt.scatter(y_pred[1:500],y[1:500], marker=".", color='b') # Plotting the predictions against the known values of the training set.

In [ ]:
tX_split_test, ind_row_groups_test, groups_mv_num_test = split_data_according_to_pattern_of_missing_values(tX_test)


''' Auxiliary function that calculates the predictions of each subgroup, joins them together in the right order
    and rescales the predictions so that they lie between -1 and 1'''
y_pred_test = predict_y_lr_weights_pattern_mv(ws_groups, tX_split_test,ind_row_groups_test,tX_test.shape[0])

In [ ]:
# The same with helper functions
DATA_TRAIN_PATH = "../data/train.csv/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# Splitting the data...
tX_split, ind_row_groups, groups_mv_num = split_data_according_to_pattern_of_missing_values(tX)
y_split = split_y_according_to_pattern_of_missing_values(y, ind_row_groups)

In [ ]:
# Adding interaction terms to the split data and calculating the optimal weights of each subgroup.
ws_groups, y_pred = find_optimal_weights_pattern_mv(tX_split_test, ind_row_groups,y_split,tX.shape[0])
plt.scatter(y_pred,y, marker=".", color='b')

In [ ]:
plt.scatter(y_pred[ind_row_groups[4]][0:800],y[ind_row_groups[4]][0:800], marker=".", color='b')

In [ ]:
DATA_TEST_PATH = "../data/test.csv/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_split_test, ind_row_groups_test, groups_mv_num_test = split_data_according_to_pattern_of_missing_values(tX_test)
y_pred_test = predict_y_lr_weights_pattern_mv(ws_groups, tX_split_test,ind_row_groups_test,tX_test.shape[0])

In [ ]:
y_pred_tX = y_pred
y_pred_test_tX = y_pred_test
ws_groups_tX = ws_groups

### Interactions between variables

In [ ]:
x = np.array([[1,2,3],[4,5,6]])
print(add_interaction_terms(x),add_square_terms(x))

In [ ]:
print(add_higher_degree_terms(x, 3))

#### Best version so far: Pattern of missing values, interactions between variables and logistic regession

#### Squares of explanatory variables

In [ ]:
DATA_TRAIN_PATH = "../data/train.csv/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# Splitting the data...
tX_split, ind_row_groups, groups_mv_num = split_data_according_to_pattern_of_missing_values(tX)
y_split = split_y_according_to_pattern_of_missing_values(y, ind_row_groups)

In [ ]:
# Adding interaction terms to the split data and calculating the optimal weights of each subgroup.
ws_groups, y_pred = find_optimal_weights_pattern_mv(add_exponential_terms_to_split_data(tX_split_test,2), ind_row_groups,y_split,tX.shape[0])
plt.scatter(y_pred,y, marker=".", color='b')

In [ ]:
plt.scatter(y_pred[ind_row_groups[4]][0:800],y[ind_row_groups[4]][0:800], marker=".", color='b')

In [ ]:
DATA_TEST_PATH = "../data/test.csv/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_split_test, ind_row_groups_test, groups_mv_num_test = split_data_according_to_pattern_of_missing_values(tX_test)
y_pred_test = predict_y_lr_weights_pattern_mv(ws_groups, add_exponential_terms_to_split_data(tX_split_test,2),ind_row_groups_test,tX_test.shape[0])

In [ ]:
y_pred_squares = y_pred
y_pred_test_squares = y_pred_test
ws_groups_squares = ws_groups

In [ ]:
len(y_pred_test_squares)

#### Cubic values of explanatory variables

In [ ]:
DATA_TRAIN_PATH = "../data/train.csv/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# Splitting the data...
tX_split, ind_row_groups, groups_mv_num = split_data_according_to_pattern_of_missing_values(tX)
y_split = split_y_according_to_pattern_of_missing_values(y, ind_row_groups)

In [ ]:
# Adding interaction terms to the split data and calculating the optimal weights of each subgroup.
ws_groups, y_pred = find_optimal_weights_pattern_mv(add_exponential_terms_to_split_data(tX_split,3), ind_row_groups,y_split,tX.shape[0])
plt.scatter(y_pred,y, marker=".", color='b')

In [ ]:
plt.scatter(y_pred[ind_row_groups[4]][0:800],y[ind_row_groups[4]][0:800], marker=".", color='b')

In [ ]:
DATA_TEST_PATH = "../data/test.csv/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_split_test, ind_row_groups_test, groups_mv_num_test = split_data_according_to_pattern_of_missing_values(tX_test)
y_pred_test = predict_y_lr_weights_pattern_mv(ws_groups, add_exponential_terms_to_split_data(tX_split_test,3),ind_row_groups_test,tX_test.shape[0])

In [ ]:
y_pred_cubic = y_pred
y_pred_test_cubic = y_pred_test
ws_groups_cubic = ws_groups

In [ ]:
groups_no_conv = np.array([1,4])

In [ ]:
y_pred_comb = y_pred_cubic
for group in groups_no_conv:
    y_pred_comb[ind_row_groups[group]] = y_pred_squares[ind_row_groups[group]]

In [ ]:
plt.scatter(y_pred_comb[ind_row_groups[1]][0:800],y[ind_row_groups[1]][0:800], marker=".", color='b')

In [ ]:
for group in groups_no_conv:
    y_pred_test[ind_row_groups_test[group]] = y_pred_test_squares[ind_row_groups_test[group]]

#### Interactions beween all variables

In [ ]:
DATA_TRAIN_PATH = "../data/train.csv/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# Splitting the data...
tX_split, ind_row_groups, groups_mv_num = split_data_according_to_pattern_of_missing_values(tX)
y_split = split_y_according_to_pattern_of_missing_values(y, ind_row_groups)

In [ ]:
# Adding interaction terms to the split data and calculating the optimal weights of each subgroup.
ws_groups, y_pred = find_optimal_weights_pattern_mv(add_interaction_terms_to_split_data(tX_split), ind_row_groups,y_split,tX.shape[0])
plt.scatter(y_pred,y, marker=".", color='b')

In [ ]:
# Adding interaction terms to the split data and calculating the optimal weights of each subgroup.
ws_groups, y_pred = find_optimal_weights_pattern_mv(add_interaction_terms_to_split_data(tX_split), ind_row_groups,y_split,tX.shape[0])
plt.scatter(y_pred,y, marker=".", color='b')

In [ ]:
plt.scatter(y_pred[ind_row_groups[4]][0:800],y[ind_row_groups[4]][0:800], marker=".", color='b')

In [ ]:
DATA_TEST_PATH = "../data/test.csv/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_split_test, ind_row_groups_test, groups_mv_num_test = split_data_according_to_pattern_of_missing_values(tX_test)
y_pred_test = predict_y_lr_weights_pattern_mv(ws_groups, add_interaction_terms_to_split_data(tX_split_test),ind_row_groups_test,tX_test.shape[0])

In [ ]:
y_pred_interact = y_pred
y_pred_test_interact = y_pred_test
ws_groups_interact = ws_groups

#### Standartization - if one uses square and cubic values they might be larger of smaller

In [ ]:
def standardize(x):

    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    
    return std_data

### !!! Overfitting

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/predicted.csv' # TODO: fill in desired name of output file for submission
tX_test, mean_x, std_x = standardize(tX_test)
tX_test  = np.c_[np.ones(tX_test.shape[0]), tX_test]
y_pred = predict_labels(w1, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
#implementer 10% du train set comme test set